# Example Exam Grade Calculations

Here, I calculate scores from a JSON file storing student responses to exam questions. See `key_and_grades.json`.

## Reading in the data and calculating raw score

In [3]:
import json

# Store student scores for some data crunching later
names = []
mult_choice = []
free_response = []

# Total points that raw scores for parts 1 & 2 are out of
total_pts_mult_choice = 30.0
total_pts_free_response = 20.0

# Which questions were hard?
missed_questions = {}

with open('key_and_grades.json', 'r') as f:
    scores = json.load(f)
    answer_key = scores['key']
    students = scores['students']
    
    for student in students:
        part1 = 0
        part2 = 0
        for key in student:
            if (key.find('q') != -1):
                if bool(student[key]) and (student[key] in (answer_key[0][key])):
                    part1 = part1 + 1
                else:
                    if key in missed_questions:
                        missed_questions[key] = missed_questions[key] + 1
                    else:
                        missed_questions[key] = 1
            elif (key.find('choice') != -1):
                    part2 = part2 + float(student[key])
                
        name = "{}, {}".format(student['lastName'], student['firstName'])
        print("{:-<28} {} ({:.2f}%)".format(name, part1, part1/total_pts_mult_choice*100.0))
        
        names.append(name)
        mult_choice.append(part1)
        free_response.append(part2)

Washington, George---------- 17 (56.67%)
Musk, Elon------------------ 29 (96.67%)
Rand, Ayn------------------- 26 (86.67%)
Polo, Marco----------------- 28 (93.33%)
Clinton, Hilary------------- 23 (76.67%)
Freud, Sigmund-------------- 25 (83.33%)
Obama, Barack--------------- 21 (70.00%)
Jones, Alex----------------- 25 (83.33%)
Uzumaki, Naruto------------- 22 (73.33%)
Skywalker, Luke------------- 19 (63.33%)
Murakami, Haruki------------ 23 (76.67%)
Mao, Zedong----------------- 28 (93.33%)
Bush, George---------------- 25 (83.33%)
Einstein, Albert------------ 29 (96.67%)
Orwell, George-------------- 28 (93.33%)
Tolstoy, Leo---------------- 15 (50.00%)
Zuckerberg, Mark------------ 28 (93.33%)
Kahlo, Frida---------------- 25 (83.33%)
West, Kanye----------------- 26 (86.67%)
von Mises, Ludwig----------- 18 (60.00%)


In [4]:
# Frequently missed questions
print(missed_questions)

{'q04': 2, 'q05': 6, 'q06': 5, 'q07': 3, 'q08': 3, 'q09': 4, 'q12': 6, 'q15': 5, 'q18': 7, 'q24': 11, 'q25': 8, 'q26': 1, 'q29': 8, 'q23': 14, 'q27': 6, 'q14': 9, 'q03': 3, 'q11': 2, 'q13': 2, 'q01': 4, 'q20': 1, 'q21': 5, 'q02': 1, 'q10': 1, 'q16': 1, 'q17': 1, 'q19': 1}


## Multiple choice

A total of 30 questions here. Some questions have more than one possible correct answer.

In [5]:
import numpy as np
mc_scores = np.asarray(mult_choice)
mc_mean = np.average(mc_scores)
mc_median = np.median(mc_scores)
mc_std = np.std(mc_scores)
print("Mean: {:.2f} ({:.2f}%)\nMedian: {:.2f} ({:.2f}%)\nStd. Dev: {:.2f}".format(mc_mean, \
    mc_mean/total_pts_mult_choice*100.0, mc_median, \
    mc_median/total_pts_mult_choice*100.0, mc_std))

Mean: 24.00 (80.00%)
Median: 25.00 (83.33%)
Std. Dev: 4.07


## Free response

Exams may include a more subjectively graded free response section.


In [6]:
fr_scores = np.asarray(free_response)
fr_mean = np.average(fr_scores)
fr_median = np.median(fr_scores)
fr_std = np.std(fr_scores)
print("Mean: {:.2f} ({:.2f}%)\nMedian: {:.2f} ({:.2f}%)\nStd. Dev: {:.2f}".format(fr_mean, 
    fr_mean/total_pts_free_response*100.0, \
    fr_median, fr_median/total_pts_free_response*100.0, fr_std))

Mean: 18.71 (93.55%)
Median: 19.83 (99.13%)
Std. Dev: 2.15


## Combined scores, first try

Weight the portion scored higher on as 70%, lower part as 30%

In [7]:
original_weighted = []

for i in range(0,len(names)):
    higher_part = 0
    lower_part = 0
    if mult_choice[i] >= free_response[i]:
        higher_part = mult_choice[i] / total_pts_mult_choice
        lower_part = free_response[i] / total_pts_free_response
    else:
        higher_part = free_response[i] / total_pts_free_response
        lower_part = mult_choice[i] / total_pts_mult_choice
        
    total = (higher_part * 0.7) + (lower_part * 0.3)
    original_weighted.append(total)
    
    print("{:-<28}: {:.1f}%".format(names[i], total*100.0))

Washington, George----------: 59.9%
Musk, Elon------------------: 97.7%
Rand, Ayn-------------------: 90.7%
Polo, Marco-----------------: 95.3%
Clinton, Hilary-------------: 76.6%
Freud, Sigmund--------------: 87.6%
Obama, Barack---------------: 78.0%
Jones, Alex-----------------: 88.3%
Uzumaki, Naruto-------------: 81.3%
Skywalker, Luke-------------: 68.6%
Murakami, Haruki------------: 83.5%
Mao, Zedong-----------------: 95.3%
Bush, George----------------: 88.1%
Einstein, Albert------------: 97.7%
Orwell, George--------------: 95.3%
Tolstoy, Leo----------------: 81.5%
Zuckerberg, Mark------------: 94.6%
Kahlo, Frida----------------: 88.0%
West, Kanye-----------------: 89.2%
von Mises, Ludwig-----------: 62.1%


## Adjustments

Here I give 20% of the exam "free". This is an easy way to adjust exam grades if students happen to score too low for a university's liking...

In [8]:
# Calculate scores
WEIGHT_FREE = 0.2 # What proportion of points are given for "free"
new_weighted = []

for i in range(0,len(names)):
    reweighted = WEIGHT_FREE + (1.0 - WEIGHT_FREE)*original_weighted[i]
    new_weighted.append(reweighted)
    print("{:-<28}: {:.1f}%".format(names[i], reweighted*100.0))


Washington, George----------: 67.9%
Musk, Elon------------------: 98.1%
Rand, Ayn-------------------: 92.5%
Polo, Marco-----------------: 96.3%
Clinton, Hilary-------------: 81.3%
Freud, Sigmund--------------: 90.1%
Obama, Barack---------------: 82.4%
Jones, Alex-----------------: 90.7%
Uzumaki, Naruto-------------: 85.0%
Skywalker, Luke-------------: 74.9%
Murakami, Haruki------------: 86.8%
Mao, Zedong-----------------: 96.3%
Bush, George----------------: 90.5%
Einstein, Albert------------: 98.1%
Orwell, George--------------: 96.3%
Tolstoy, Leo----------------: 85.2%
Zuckerberg, Mark------------: 95.7%
Kahlo, Frida----------------: 90.4%
West, Kanye-----------------: 91.3%
von Mises, Ludwig-----------: 69.7%


In [9]:
final_scores = np.asarray(new_weighted)
final_mean = np.average(final_scores)
final_median = np.median(final_scores)
final_std = np.std(final_scores)
print("Mean: {:.2f}\nMedian: {:.2f}\nStd. Dev: {:.2f}".format(final_mean*100, \
    final_median*100, final_std*100))

Mean: 87.97
Median: 90.46
Std. Dev: 8.74
